In [3]:
import spotipy
import spotipy.util as util
import time
import os

In [4]:
#Auth into Spotify api
secret = os.environ.get('SPOTIPY_CLIENT_SECRET')
scope = 'playlist-modify-private playlist-modify-public user-top-read playlist-read-private user-library-read user-follow-read'
username = 'isaacbradys'
token = util.prompt_for_user_token(username, scope,
                                   client_id="569563cc71bb4548a49d5e53d7b226a3", 
                                   client_secret = secret,
                                   redirect_uri = 'http://localhost:8080')
spotify = spotipy.Spotify(auth=token)

In [3]:
#Grab the artists I follow into list
following_results = spotify.current_user_followed_artists(limit=50)
artist_names = []
artist_pop = []
NextPage = True
while NextPage:
    if not following_results['artists']['next']:
        NextPage = False
    for item in following_results['artists']["items"]:
        artist_names.append(item['id'])
        artist_pop.append(item['popularity'])
    following_results = spotify.next(following_results['artists'])
    
following_count = len(artist_names)
print("Following " + str(following_count) + 
      " artists. They're average popularity is " + str(round(sum(artist_pop)/len(artist_pop), 2)) + " out of 100")

Following 53 artists. They're average popularity is 52.34 out of 100


In [4]:
#Grab my Top 20 Current Artists 
top_artists_results = spotify.current_user_top_artists()
top_artists_pop = []

for item in top_artists_results["items"]:
    print(item["name"])
    artist_names.append(item['id'])
    top_artists_pop.append(item['popularity'])
    
print("Out of my top 20 artists. They're average popularity is " + 
      str(round(sum(top_artists_pop)/len(top_artists_pop), 2)) + " out of 100")

Mac DeMarco
Tame Impala
Alexandra Savior
Andy Shauf
Men I Trust
Kevin Morby
Dirty Projectors
Jim Reeves
Spock's Grand Amen
Stella Donnelly
WILLOW
Isobel Waller-Bridge
Chaos Chaos
Her's
Kate Bollinger
Faye Webster
Oscar Lang
Anna Burch
Mac Miller
Alvvays
Out of my top 20 artists. They're average popularity is 56.75 out of 100


In [5]:
#Remove duplicates from artists and follow and my top 20
artist_names = list(dict.fromkeys(artist_names))
print(len(artist_names))

60


In [6]:
#Grab the top 10 tracks from the 3 most related artist of each artist I follow
similar_artist = []
songs = []

for item in artist_names:
    results = spotify.artist_related_artists(item)
    count=0
    for artist in results['artists']:
        similar_artist.append(artist['name'])
        if artist['id'] in artist_names:
            continue
        songs.append((spotify.artist_top_tracks(artist['id'], country='US'))['tracks'])
        count+=1
        if count == 3:
            break

In [7]:
#Put song names and ids into list
final_song_list = []

for song in songs:
    for s in song:
        final_song_list.append((s['name'], s['id']))

In [8]:
#Eliminate duplicates in song list
print("Started with " + str(len(final_song_list)) + " and now we have: ")
final_song_list = list(dict.fromkeys(final_song_list))
print(len(final_song_list))

Started with 1711 and now we have: 
1308


In [ ]:
#Put tracks into playlist - wait 5 seconds after every 10 songs so don't hit API limit (takes like 13 minutes)
print("Script found " + str(len(final_song_list)) + " songs from similar artists!")
stall = 0
for track_id in final_song_list:
    spotify.user_playlist_add_tracks("isaacbradys","1nSh4V8S6Zuzj2f0HpRzOB", [track_id[1]])
    stall+=1
    if stall%10 == 0:
        time.sleep(5)
print("These songs were added! Go check them out!")

Script found 1308 songs from similar artists!
